In [ ]:
#oandapyV20を使って為替データを取得
#https://qiita.com/StingQian/items/750ab38f468bdbd2f458

import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import pandas as pd

access_token = "f98bf979d013f7775db89f84edd7f2c9-ed0a298fd59d5090f45471641b6d9842" #自身のaccess_tokenを記載
accountID = "101-009-23770133-001" #自身のaccountIDを記載
api = oandapyV20.API(access_token=access_token)

# 1時間間隔で500データ（ドル円）を取得
r = instruments.InstrumentsCandles(instrument="USD_JPY", params={"count": 500, "granularity": "H1"})
api.request(r)
#r.response['candles'][1]['mid']["c"]

In [ ]:
#【oandapyV20】OandaAPIで価格をStreaming配信で取得する
#https://blog.np-sys.com/oandaapi-get-streaming/

#外部モジュールの定義
import json
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
    
#口座情報 (自分で取得した情報と置き換えてください。）
API_TOKEN = 'f98bf979d013f7775db89f84edd7f2c9-ed0a298fd59d5090f45471641b6d9842'
API_ID = '101-009-23770133-001'
  
#通貨ペア
INSTRUMENT = "USD_JPY"
  
api = API(API_TOKEN)
  
try:
        #最新レートの取得💡
        import oandapyV20.endpoints.pricing as pricing
        params = {"instruments" : INSTRUMENT}
        request = pricing.PricingInfo(accountID = API_ID, params = params)
        response_body = api.request(request)
        print("最新レート")
        print(json.dumps(response_body, indent=2))
  
        #ろうそく足の取得💡
        import oandapyV20.endpoints.instruments as instruments
        params = {
                "price" : "B",
                "granularity" : "M5",
                "count" : 10,
                "smooth" : False,
                }
        request = instruments.InstrumentsCandles(instrument=INSTRUMENT, params = params)
        response_body = api.request(request)
        print("ろうそく足")
        print(json.dumps(response_body, indent=2))
  
        #口座情報（通貨ペア別）の取得💡
        import oandapyV20.endpoints.accounts as accounts
        params = {"instruments" : INSTRUMENT}
        request = accounts.AccountInstruments(API_ID, params=params)
        response_body = api.request(request)
        print("口座情報（通貨ペア別）")
        print(json.dumps(response_body, indent=2))
  
        #スプレッド💡
        import oandapyV20.endpoints.forexlabs as labs
        params = {"instrument" : INSTRUMENT, "period" : 3600}
        request = labs.Spreads(params = params)
        response_body = api.request(request)
        print("スプレッド")
        print(json.dumps(response_body, indent=2))
    
        #ろうそく足の取得拡張版💡
        from oandapyV20.contrib.factories import InstrumentsCandlesFactory as icf
        import pandas as pd
          
        params = {
                "granularity" : "H1",
                "count" : 5000,
                "from" : "2010-01-01T00:00:00Z",
                "to" : "2020-12-31T00:00:00Z"              
                }
  
        filename_json = "C:/tmp/candle.json"
        filename_csv = "C:/tmp/candle.csv"
          
        df = pd.DataFrame(index=[])
        for r in icf(instrument=INSTRUMENT, params=params):
                api.request(r)
                record = pd.json_normalize(r.response.get('candles'))
                df = df.append(record, ignore_index=True)    #データフレームへの変換
    
        #ファイル書き込み  
        df.to_json(filename_json, orient='records')
        df.to_csv(filename_csv, sep=",")
  
        #ファイル読み込み
        with open(filename_json, 'r') as IN:
                data=IN.read()
        data = json.loads(data)
        i = len(data)
        print("ろうそく足拡張")
        print("配列の要素数: %s" %i)
        print("頭: %s" %data[0]['time'])
        print("お尻: %s" %data[i-1]['time'])   
  
#エラー時の処理💡
except V20Error as e:
        print('V20Error!=>:\n %s' %e)
  
except Exception as e:
        print('Error!=>\n%s' %e)